In [1]:
import numpy as np
import pandas as pd
import json
import dill         # 0.3.2
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

## Set Global/Environment Variables

In [2]:
model_name = "GEMM_STREAM_VMID=4034"

## Set Thresholds

In [3]:
gpa_threshold = 36
rip_threshold = 3

## Load Dataset/Static Param List

In [4]:
original_dataset = pd.concat([pd.read_csv("../로그 데이터/GEMM_STREAM/GEMM_STREAM/gemm_stream_generic_generic2_{}.csv".format(i), dtype=np.object) for i in range(1, 10)], axis=0)
original_dataset = original_dataset[original_dataset["vmid"] == '4034']
original_dataset

,extra,time,address_type,gpa,rip,vmid
0,[861367.505406],05:46:34:647695,PF,988827648,18446744072452043863,4034
1,[861367.505455],05:46:34:647744,PF,705122304,18446744072452043863,4034
2,[861367.505459],05:46:34:647748,PF,705122304,18446744072452043863,4034
3,[861367.505464],05:46:34:647754,PF,1227321344,18446744072452043863,4034
4,[861367.505468],05:46:34:647757,PF,1227321344,18446744072452043863,4034
...,...,...,...,...,...,...
407190,[957446.648224],08:27:53:789007,PF,253842912,18446744072451347936,4034
407191,[957446.648252],08:27:53:789035,GPA,878240226,18446744072637686242,4034
407192,[957446.650168],08:27:53:790949,GPA,878245255,18446744072637683079,4034
407193,[957446.650178],08:27:53:790962,GPA,878245319,18446744072637683143,4034


In [5]:
original_dataset = original_dataset[["gpa", "rip"]].dropna()       # rip for PCs
original_dataset

,gpa,rip
0,988827648,18446744072452043863
1,705122304,18446744072452043863
2,705122304,18446744072452043863
3,1227321344,18446744072452043863
4,1227321344,18446744072452043863
...,...,...
407190,253842912,18446744072451347936
407191,878240226,18446744072637686242
407192,878245255,18446744072637683079
407193,878245319,18446744072637683143


## Dataset Processing Functions

In [6]:
class CalculateDelta(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        X_transformed = X[:-1] - X[1:]
        # In case of unsigned types, change its type to string type
        if X_transformed.dtype in [np.uint8, np.uint16, np.uint32, np.uint64]:
            X_transformed = X_transformed.astype(np.string_)
        return X_transformed

    def inverse_transform(self, X, y=None):     # Just for test_pipeline.inverse_transform()
        return X

In [7]:
class NoiseTokenizer(TransformerMixin):
    def __init__(self, minimum_category_occurence=2, oov_token=-1):        
        self.minimum_category_occurence = minimum_category_occurence
        self.oov_token = oov_token
    
    def fit(self, X, y=None):
        if X.dtype == np.string_:
            self.oov_token = str(self.oov_token)

        mask = (pd.Series(X).value_counts() <= self.minimum_category_occurence)
        noise_index = np.where(np.isin(X, mask.index[mask == True]))[0]
    
        X[noise_index] = self.oov_token
        return self
    
    def transform(self, X, y=None):
        return X

    def inverse_transform(self, X, y=None):
        return X

In [8]:
class SparseCategoryEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, oov_token=-1):
        self.oov_token = oov_token

    def fit(self, X, y=None):
        X_counts = pd.Series(X).value_counts()
        self.vocab_size = len(X_counts)
        self.word_index = X_counts.index
        
        if X.dtype in [np.dtype("S" + str(i)) for i in range(24)]:  # X.dtype == |S{0~24}
            # As np.string_ type is byte type, not str(), need to be decoded.
            self.vocabulary = {X_counts.index[i].decode():i for i in range(self.vocab_size)}
        else:
            self.vocabulary = {X_counts.index[i]:i for i in range(self.vocab_size)}
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        if X.dtype in [np.dtype("S" + str(i)) for i in range(24)]:
            self.oov_token = str(self.oov_token)
            for i in range(len(X)):
                if X[i] in self.word_index:
                    X_transformed.append(self.vocabulary[X[i].decode()])
                else:
                    X_transformed.append(self.vocabulary[self.oov_token])
        else:
            for i in range(len(X)):
                if X[i] in self.word_index:
                    X_transformed.append(self.vocabulary[X[i]])
                else:
                    X_transformed.append(self.vocabulary[self.oov_token])

        return np.array(X_transformed)

    def inverse_transform(self, X, y=None):
        return np.array([self.word_index[X[i]] for i in range(len(X))])

## Process Train/Validation Dataset

In [9]:
# Train / Val / Test Ratio : 70% / 15% / 15%
train_val_set, test_set = train_test_split(original_dataset, test_size=0.15, shuffle=False)
#train_set, val_set = train_test_split(train_val_set, test_size=0.2, shuffle=False)

In [10]:
train_val_gpa = train_val_set["gpa"].values.astype(np.int64)
train_val_rip = train_val_set["rip"].values.astype(np.uint64)      # As uint64 not exists in pandas

test_gpa = test_set["gpa"].values.astype(np.int64)
test_rip = test_set["rip"].values.astype(np.uint64)      # As uint64 not exists in pandas

In [11]:
gpa_train_pipeline = Pipeline([
    ('calculate_delta', CalculateDelta()),
    ('noise_tokenizer', NoiseTokenizer(minimum_category_occurence=gpa_threshold)),
    ('sparse_category_encoder', SparseCategoryEncoder())
])

rip_train_pipeline = Pipeline([
    ('calculate_delta', CalculateDelta()),
    ('noise_tokenizer', NoiseTokenizer(minimum_category_occurence=rip_threshold)),
    ('sparse_category_encoder', SparseCategoryEncoder())
])

In [12]:
processed_train_val_gpa = gpa_train_pipeline.fit_transform(train_val_gpa.copy())
processed_train_val_gpa

array([1, 0, 1, ..., 0, 1, 0])

In [13]:
processed_train_val_rip = rip_train_pipeline.fit_transform(train_val_rip.copy())
processed_train_val_rip

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
pd.Series(processed_train_val_gpa).value_counts(), pd.Series(processed_train_val_rip).value_counts()

(0      650130
 1      634931
 2      154000
 3       10662
 4       10661
         ...  
 241        37
 236        37
 235        37
 234        37
 242        37
 Length: 243, dtype: int64,
 0       1303291
 1         27937
 2         11004
 3         10625
 4         10558
          ...   
 2306          4
 2688          4
 2689          4
 2821          4
 2499          4
 Length: 2825, dtype: int64)

In [15]:
pd.Series(processed_train_val_gpa).value_counts().value_counts()[:10], pd.Series(processed_train_val_rip).value_counts().value_counts()[:10]

(43    11
 40    11
 37     9
 48     9
 41     8
 38     7
 46     5
 45     5
 42     5
 67     4
 dtype: int64,
 4     580
 5     332
 6     265
 7     179
 8     155
 9     128
 10    116
 11    103
 13     75
 12     59
 dtype: int64)

In [16]:
processed_train_gpa, processed_val_gpa = train_test_split(processed_train_val_gpa, test_size=0.2, shuffle=False)
processed_train_rip, processed_val_rip = train_test_split(processed_train_val_rip, test_size=0.2, shuffle=False)

In [17]:
processed_train_gpa.shape, processed_val_gpa.shape, processed_train_rip.shape, processed_val_rip.shape  # check 

((1254014,), (313504,), (1254014,), (313504,))

In [18]:
train_gpa = train_val_gpa[:processed_train_gpa.shape[0]+1]
val_gpa = train_val_gpa[processed_train_gpa.shape[0]:]

train_rip = train_val_rip[:processed_train_rip.shape[0]+1]
val_rip = train_val_rip[processed_train_rip.shape[0]:]

In [19]:
train_gpa.shape, val_gpa.shape, train_rip.shape, val_rip.shape

((1254015,), (313505,), (1254015,), (313505,))

## Process Test Dataset

In [20]:
processed_test_gpa = gpa_train_pipeline.transform(test_gpa)
processed_test_gpa

array([ 0,  1,  0, ..., 18, 13, 14])

In [21]:
processed_test_rip = rip_train_pipeline.transform(test_rip)
processed_test_rip

array([ 0,  0,  0, ..., 41, 34, 33])

In [22]:
dict(list(rip_train_pipeline["sparse_category_encoder"].vocabulary.items())[:10])

{'0': 0,
 '-1': 1,
 '94268222548486': 2,
 '18446649805487003130': 3,
 '333': 4,
 '18446744073709551579': 5,
 '37': 6,
 '18446744073709551587': 7,
 '18446649485997979380': 8,
 '94587711572236': 9}

## Concat GPA and RIP

In [23]:
train_set = np.c_[train_gpa, train_rip]
val_set = np.c_[val_gpa, val_rip]
test_set = np.c_[test_gpa, test_rip]

processed_train_set = np.c_[processed_train_gpa, processed_train_rip]
processed_val_set = np.c_[processed_val_gpa, processed_val_rip]
processed_test_set = np.c_[processed_test_gpa, processed_test_rip]
train_set, val_set, test_set

(array([[9.88827648e+08, 1.84467441e+19],
        [7.05122304e+08, 1.84467441e+19],
        [7.05122304e+08, 1.84467441e+19],
        ...,
        [4.76680192e+08, 9.42669650e+13],
        [4.42085376e+08, 9.42669650e+13],
        [4.42085376e+08, 9.42669650e+13]]),
 array([[4.42085376e+08, 9.42669650e+13],
        [3.78806272e+08, 9.42669650e+13],
        [3.78806272e+08, 9.42669650e+13],
        ...,
        [1.60419430e+09, 1.84467441e+19],
        [1.43124070e+09, 1.84467441e+19],
        [1.43124070e+09, 1.84467441e+19]]),
 array([[1.40674253e+09, 1.84467441e+19],
        [1.40674253e+09, 1.84467441e+19],
        [1.62596864e+09, 1.84467441e+19],
        ...,
        [8.78245255e+08, 1.84467441e+19],
        [8.78245319e+08, 1.84467441e+19],
        [8.78245368e+08, 1.84467441e+19]]))

In [24]:
train_set = pd.DataFrame(train_set, columns=["gpa", "rip"], index=None)
val_set = pd.DataFrame(val_set, columns=["gpa", "rip"], index=None)
test_set = pd.DataFrame(test_set, columns=["gpa", "rip"], index=None)

processed_train_set = pd.DataFrame(processed_train_set, columns=["gpa", "rip"], index=None)
processed_val_set = pd.DataFrame(processed_val_set, columns=["gpa", "rip"], index=None)
processed_test_set = pd.DataFrame(processed_test_set, columns=["gpa", "rip"], index=None)
train_set

,gpa,rip
0,9.888276e+08,1.844674e+19
1,7.051223e+08,1.844674e+19
2,7.051223e+08,1.844674e+19
3,1.227321e+09,1.844674e+19
4,1.227321e+09,1.844674e+19
...,...,...
1254010,4.129546e+08,9.426697e+13
1254011,4.766802e+08,9.426697e+13
1254012,4.766802e+08,9.426697e+13
1254013,4.420854e+08,9.426697e+13


## Save Processed Datasets

In [25]:
# Original 
train_set.to_csv("data/{}_train_set_original.csv".format(model_name), index=None)
val_set.to_csv("data/{}_val_set_original.csv".format(model_name), index=None)
test_set.to_csv("data/{}_test_set_original.csv".format(model_name), index=None)

# Processed 
processed_train_set.to_csv("data/{}_train_set.csv".format(model_name), index=None)
processed_val_set.to_csv("data/{}_val_set.csv".format(model_name), index=None)
processed_test_set.to_csv("data/{}_test_set.csv".format(model_name), index=None)

## Save Pipeline/Statics

In [26]:
with open("static/pipeline_gpa.pkl", 'wb') as f:
    dill.dump(gpa_train_pipeline, f)

with open("static/pipeline_rip.pkl", 'wb') as f:
    dill.dump(rip_train_pipeline, f)

In [27]:
np.savetxt("static/vocabulary_gpa.csv", np.array(list(gpa_train_pipeline["sparse_category_encoder"].vocabulary.keys())), fmt="%d", delimiter="\n")

np.savetxt("static/vocabulary_rip.csv", np.array(list(rip_train_pipeline["sparse_category_encoder"].vocabulary.keys())), fmt="%s", delimiter="\n")

In [28]:
dict(list(gpa_train_pipeline["sparse_category_encoder"].vocabulary.items())[:20])

{0: 0,
 -1: 1,
 -4096: 2,
 -643931498: 3,
 -643931494: 4,
 643931494: 5,
 643931498: 6,
 -643931490: 7,
 643931490: 8,
 333: 9,
 -12288: 10,
 -8192: 11,
 -333: 12,
 -64: 13,
 -49: 14,
 638771200: 15,
 -638771200: 16,
 -24576: 17,
 -5029: 18,
 -16384: 19}

In [29]:
dict(list(rip_train_pipeline["sparse_category_encoder"].vocabulary.items())[:20])

{'0': 0,
 '-1': 1,
 '94268222548486': 2,
 '18446649805487003130': 3,
 '333': 4,
 '18446744073709551579': 5,
 '37': 6,
 '18446744073709551587': 7,
 '18446649485997979380': 8,
 '94587711572236': 9,
 '29': 10,
 '18446649485997979597': 11,
 '94587711572019': 12,
 '94268222548265': 13,
 '18446744073709551612': 14,
 '18446649805487003351': 15,
 '4': 16,
 '18446744073703730794': 17,
 '5820489': 18,
 '94587711572486': 19}

In [30]:
gpa_train_pipeline["sparse_category_encoder"].vocab_size, rip_train_pipeline["sparse_category_encoder"].vocab_size

(243, 2825)